# Miller Predicting a Continuous Target with Regression

**Author:** Dan Miller

**Date:** November 10th, 2025

**Objective:** Be able to predict a continuous numeric target, fare, using features in the Titanic dataset

## Introduction

This project explores the seaborn Titanic dataset.  With the goal of being able to predict a continuous target with regression, linear regression as well as some alternative models will all be created and evaluated in terms of predicting fare, the price of the journey.  The data will be explored and prepped before the models are created.  There will be a summary at the end to discuss the findings.

## Imports

In [1]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import ElasticNet, LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
